## Introduction to Digital Earth Australia

#### [Digital Earth Australia](https://www.ga.gov.au/dea) (DEA) is a digital platform that offer large amounts of Earth observation data covering continental Australia and ready to be analysed.

### Data Sources

#### Digital Earth Australia catalogues data from a range of satellite sensors. The earliest datasets of optical satellite imagery in DEA date from 1986. DEA includes data from:

* [Landsat 5 TM](https://www.usgs.gov/land-resources/nli/landsat/landsat-5?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS5 TM), operational between March 1984 and January 2013
* [Landsat 7 ETM+](https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS7 ETM+), operational since April 1999
* [Landsat 8 OLI](https://www.usgs.gov/land-resources/nli/landsat/landsat-8?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS8 OLI), operational since February 2013
* [Sentinel 2A MSI](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) (S2A MSI), operational since June 2015
* [Sentinel 2B MSI](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) (S2B MSI, operational since March 2017

#### Landsat missions are jointly operated by the United States Geological Survey (USGS) and National Aeronautics and Space Administration (NASA). Sentinel missions are operated by the European Space Agency (ESA). One major difference between the two programs is the spatial resolution: each Landsat pixel represents 30 x 30 m on the ground while each Sentinel-2 pixel represents 10 x 10 m to 60 x 60 m depending on the spectral band.

![Image](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/dmidS2LS7Comparison.png)


#### Previous figure shows that the numbering of the bands is specific to each sensor and has no relationship with its postion in the spectrum.

### Analysis Ready Data

#### Digital Earth Australia produces Analysis Ready Data (ARD) for each of the sensors listed above. The [ARD standard](http://ceos.org/ard/) for satellite data requires that data have undergone a number of processing steps, along with the creation of additional attributes for the data. DEA's ARD datasets include the following characteristics:

* **Geometric correction:** This includes establishing ground position, accounting for terrain (orthorectification) and ground control points, and assessing absolute position accuracy. 
Geometric calibration means that imagery is positioned accurately on the Earth's surface and stacked consistently so that sequential observations can be used to track meaningful change over time.
Adjustments for ground variability typically use a Digital Elevation Model (DEM).
* **Surface reflectance correction:** This includes adjustments for sensor/instrument gains, biases and offsets, include adjustments for terrain illumination and sensor viewing angle with respect to the pixel position on the surface.
Once satellite data is processed to surface reflectance, pixel values from the same sensor can be compared consistently both spatially and over time.
* **Observation attributes:** Per-pixel metadata such as quality flags and content attribution that enable users to make informed decisions about the suitability of the products for their use. For example, clouds, cloud shadows, missing data, saturation and water are common pixel level attributes.
* **Metadata:** Dataset metadata including the satellite, instrument, acquisition date and time, spatial boundaries, pixel locations, mode, processing details, spectral or frequency response and grid projection.


#### Optical sensors, such as those on the Landsat and Sentinel-2 satellites, measure light that has come from the sun and been reflected by the Earth's surface. The sensor measures the intensity of light in each of its spectral bands (known as "radiance"). The intensity of this light is affected by many factors including the angle of the sun relative to the ground, the angle of the sensor relative to the ground, and how the light interacts with the Earth's atmosphere on its way to the sensor.  Because radiance can be affected by so many factors, it is typically more valuable to determine how much light was originally reflected at the ground level. This is known as bottom-of-atmosphere **surface reflectance**. Surface reflectance can be calculated by using robust physical models to correct the observed radiance values based on atmospheric conditions, the angle of the sun, sensor geometry and local topography or terrain.

#### There are many approaches to satellite surface reflectance correction and DEA opts to use two: NBAR and NBART.

* **NBAR**: NBAR stands for *Nadir-corrected BRDF Adjusted Reflectance*, where BRDF stands for *Bidirectional reflectance distribution function*. The approach involves atmospheric correction to compute bottom-of-atmosphere radiance, and bi-directional reflectance modelling to remove the effects of topography and angular variation in reflectance. NBAR can be useful for analyses in extremely flat areas not affected by terrain shadow, and for producing attractive data visualisations that are not affected by NBART's nodata gaps (see below).

* **NBART**: NBART has the same features of NBAR but includes an additional *terrain illumination* reflectance correction that takes into account the surface topography. Terrain affects optical satellite images in a number of ways; for example, slopes facing the sun receive more sunlight and appear brighter compared to those facing away from the sun. This correction is performed with a Digital Surface Model (DSM). NBART is typically the default choice for performing analysis that are consistent through time. However, it can be introduce distortion in flat areas due to the noise in DSM data.

![Comparison between NBAR and NBART](data/nbar_nbart_animation.gif)

#### The animation above demonstrates how the NBART correction results in a significantly more two-dimensional looking image that is less affected by terrain illumination and shadow. Black pixels in the NBART image represent areas of deep terrain shadow that can't be corrected as they're determined not to be viewable by either the sun or the satellite. These are represented by -999 `nodata` values in the data.

> Note: Remember that remote sensing data is often stored as `uint16` to save space and `NaN` is only available for floating point types, that is why we need a value `-999` in this case to designate not-a-number data.

#### The *Observation Attributes (OA)* are a suite of measurements included in DEA's analysis ready datasets. They are an assessment of each image pixel to determine if it is an unobscured, unsaturated observation of the Earth's surface, along with whether the pixel is represented in each spectral band. The OA product allows users to exclude pixels that do not meet the quality criteria for their analysis. The capacity to automatically exclude such pixels is essential for analysing any change over time, since poor-quality pixels can significantly alter the percieved change over time. The most common use of OA is for cloud masking, where users can choose to remove images that have too much cloud, or ignore the clouds within each satellite image. A demonstration of how to use cloud masking can be found in the [masking data](../Frequently_used_code/Masking_data.ipynb) notebook.

#### The OA suite of measurements include the following observation pixel-based attributes:

* Null pixels
* Clear pixels
* Cloud pixels
* Cloud shadow pixels
* Snow pixels
* Water pixels
* Terrain shaded pixels
* Spectrally contiguous pixels (i.e. whether a pixel contains data in every spectral band)

#### Also included is a range of pixel-based attributes related to the satellite, solar and sensing geometries:

* Solar zenith
* Solar azimuth
* Satellite view
* Incident angle
* Exiting angle
* Azimuthal incident
* Azimuthal exiting
* Relative azimuth
* Timedelta


#### DEA uses XArray as its underlying data model and defines a simple querying interface to select satellite data which returns as `Dataset` objects.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import datacube

dc = datacube.Datacube(app='welcome-to-dea')

type(dc)

#### This `dc` object is the handle to all DEA's functionality. For example to list the products available:

In [ ]:
dc.list_products().head()

#### Still, to select data we have to use numpy indexes. Wouldn't it be convenient to be able to select data from the images using the coordinates of the pixels instead of their relative positions?

#### This is exactly what XArray solves! Let's see how it works:

In [ ]:
query={'lat': (-35.25, -35.35),
         'lon': (149.05, 149.17),
         'time':('2010-01-01', '2019-01-15')}

ds = dc.load(product='ls8_nbart_geomedian_annual', **query)

ds

#### To explore XArray we have a file containing some reflectance data of Canberra that has been generated using the DEA library.

#### The object that we get `ds` is a XArray `Dataset`, which in some ways is very similar to the dictionary that we created before, but with lots of convenient functionality available.

In [ ]:
ds[['red', 'green', 'blue']].isel(time=0).to_array().plot.imshow(robust=True, figsize=(6,6))

#### Now that we have access to any location in Australia, select any place that you like in Australia and create a NRG→RGB False colour image, where N is the 'near-infrared' `nir` channel.

_Be careful with the extents in your request. Cubes with sizes 0.1 degree are easily managed by the Sandbox but you can quickly run out of memory if you increase this extent._